<a href="https://colab.research.google.com/github/anathnath/PythonProject2020/blob/master/covid19DataAnalysis3rdSeptember2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
from shapely.geometry import  Point, Polygon
import pandas.util.testing as tm

In [ ]:
!pip install geopandas

In [ ]:
confirmedlink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deathlink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recoveredlink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
countrylink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"


In [ ]:
confirmed_df = pd.read_csv(confirmedlink)
print(confirmed_df.shape)
deaths_df = pd.read_csv(deathlink)
print(deaths_df.shape)
recovered_df = pd.read_csv(recoveredlink)
print(recovered_df.shape)
cases_country_df = pd.read_csv(countrylink)
print(cases_country_df.shape)

In [ ]:
confirmed_df = confirmed_df.replace(np.nan, '', regex=True)
deaths_df = deaths_df.replace(np.nan, '', regex=True)
recovered_df = recovered_df.replace(np.nan, '', regex=True)
cases_country_df = cases_country_df.replace(np.nan, '', regex=True)

In [ ]:
global_data = cases_country_df.copy().drop(['Lat','Long_','Country_Region','Last_Update'],axis =1)
global_summary = pd.DataFrame(global_data.sum()).transpose()

In [ ]:
confirmed_ts = confirmed_df.copy().drop(['Lat','Long','Country/Region','Province/State'],axis =1)
confirmed_ts_summary = confirmed_ts.sum()

In [ ]:
fig_1 = go.Figure(data=go.Scatter(x=confirmed_ts_summary.index, y = confirmed_ts_summary.values, mode='lines+markers'))
fig_1.update_layout(title='Total Coronavirus Confirmed Cases (Globally)',
                  yaxis_title='Confirmed Cases', xaxis_tickangle = 315 )
fig_1.show()

In [ ]:
confirmed_ts_summary

In [ ]:
# Initializing Color Array to be used across the analysis
color_arr = px.colors.qualitative.Dark24

In [ ]:
def draw_plot(ts_array, ts_label, title, colors, mode_size, line_size, x_axis_title , y_axis_title, tickangle = 0, yaxis_type = "", additional_annotations=[]):
    # initialize figure
    fig = go.Figure()
    # add all traces
    for index, ts in enumerate(ts_array):
        fig.add_trace(go.Scatter(x=ts.index,
                                 y = ts.values,
                                 name = ts_label[index],
                                 line=dict(color=colors[index], width=line_size[index]),connectgaps=True,))
    # base x_axis prop.
    x_axis_dict = dict(showline=True, 
                       showgrid=True, 
                       showticklabels=True, 
                       linecolor='rgb(204, 204, 204)', 
                       linewidth=2,
                       ticks='outside',
                       tickfont=dict(family='Arial',size=12,color='rgb(82, 82, 82)',))
    # setting x_axis params
    if x_axis_title:
        x_axis_dict['title'] = x_axis_title
    
    if tickangle > 0:
        x_axis_dict['tickangle'] = tickangle
    
    # base y_axis prop.
    y_axis_dict = dict(showline = True,
                       showgrid = True,
                       showticklabels=True,
                       linecolor='rgb(204, 204, 204)',
                       linewidth=2,)
    # setting y_axis params
    if yaxis_type != "":
        y_axis_dict['type'] = yaxis_type
    
    if y_axis_title:
        y_axis_dict['title'] = y_axis_title
        
#     # uncomment legend if you want to move the legend around
#     legend= dict(x=1,
#                  y=1,
#                  traceorder="normal",
#                  font=dict(family="sans-serif",size=12,color="black"),
#                  bgcolor="LightSteelBlue",
#                  bordercolor="Black",
#                  borderwidth=2)

#updating the layout
    fig.update_layout(xaxis = x_axis_dict,
                      yaxis = y_axis_dict,
                      autosize=True,
                      margin=dict(autoexpand=False,l=100,r=20,t=110,),
                      showlegend=True,
#                       legend = legend
                     )

    # base annotations for any graph
    annotations = []
    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05, xanchor='left', yanchor='bottom',
                                  text=title,
                                  font=dict(family='Arial',size=16,color='rgb(37,37,37)'),showarrow=False))
    # adding annotations in params
    if len(additional_annotations) > 0:
        annotations.append(additional_annotations)
    
    #updating the layout
    fig.update_layout(annotations=annotations)

    return fig

In [ ]:
confimed_agg_ts = confirmed_df.copy().drop(['Lat','Long','Country/Region','Province/State'],axis =1).sum()
death_agg_ts = deaths_df.copy().drop(['Lat','Long','Country/Region','Province/State'],axis =1).sum()
recovered_agg_ts = recovered_df.copy().drop(['Lat','Long','Country/Region','Province/State'],axis =1).sum()

#There is no timeseries data for Active cases, therefore it needs to be engineered separately
active_agg_ts = pd.Series(
    data=np.array(
        [x1 - x2 - x3  for (x1, x2, x3) in zip(confimed_agg_ts.values,death_agg_ts.values, recovered_agg_ts.values)]),
    index= confimed_agg_ts.index)

#Plot and add traces for all the aggregated timeseries

In [ ]:
ts_array = [confimed_agg_ts, active_agg_ts, recovered_agg_ts, death_agg_ts]
labels = ['Confirmed', 'Active', 'Recovered', 'Deaths']
colors = [color_arr[5],  color_arr[0], color_arr[2], color_arr[3]]
mode_size = [8, 8, 12, 8]
line_size = [2, 2, 4, 2]

# Calling the draw_plot function defined above
fig_2 = draw_plot(ts_array = ts_array, 
                  ts_label = labels , 
                  title = "Covid-19 Case Status(22nd Jan to 2nd September 2020)",
                  colors = colors, mode_size = mode_size, 
                  line_size = line_size , 
                  x_axis_title = "Date" , 
                  y_axis_title = "Case Count", 
                  tickangle = 90, 
                  yaxis_type = "", additional_annotations=[])
fig_2.show()


In [ ]:
confirmed_India_ts = confirmed_df[confirmed_df['Country/Region']=="India"]
confirmed_India_ts = confirmed_India_ts.drop(
    ['Lat','Long','Country/Region','Province/State'],axis =1).reset_index(drop=True).sum()

deaths_India_ts = deaths_df[deaths_df['Country/Region']=="India"]
deaths_India_ts = deaths_India_ts.drop(
    ['Lat','Long','Country/Region','Province/State'],axis =1).reset_index(drop=True).sum()

recovered_India_ts = recovered_df[recovered_df['Country/Region']=="India"]
recovered_India_ts = recovered_India_ts.drop(
    ['Lat','Long','Country/Region','Province/State'],axis =1).reset_index(drop=True).sum()

active_India_ts = pd.Series(
    data=np.array(
        [x1 - x2 - x3  for (x1, x2, x3) in zip(
            confirmed_India_ts.values,deaths_India_ts.values, recovered_India_ts.values)
        ] 
    ), 
    index= confirmed_India_ts.index
)

In [ ]:
confirmed_India_ts

#Different Cases

In [ ]:
confirmed=pd.read_csv(confirmedlink)
recovered=pd.read_csv(recoveredlink)
death=pd.read_csv(deathlink)
countrywise=pd.read_csv(countrylink)

In [ ]:
confirmed = confirmed.replace(np.nan, '', regex=True)
death = death.replace(np.nan, '', regex=True)
recovered = recovered.replace(np.nan, '', regex=True)
countrywise = countrywise.replace(np.nan, '', regex=True)
countrywise = countrywise.replace(np.NaN, '', regex=True)

In [ ]:
confirmed=confirmed.drop(['Province/State','Lat','Long'], axis=1)
death=death.drop(['Province/State','Lat','Long'], axis=1)
recovered=recovered.drop(['Province/State','Lat','Long'], axis=1)

In [ ]:
confirmed=confirmed.groupby(confirmed['Country/Region']).sum()
death=death.groupby(death['Country/Region']).sum()
recovered=recovered.groupby(recovered['Country/Region']).sum()


In [ ]:
confirmed=confirmed.T
death=death.T
recovered=recovered.T


In [ ]:
new_cases=confirmed.copy()

for day in range(1,len(confirmed)):
    new_cases.iloc[day]=confirmed.iloc[day]-confirmed.iloc[day-1]
    
growth_rate=confirmed.copy()
for day in range(1,len(confirmed)):
    growth_rate.iloc[day]=(new_cases.iloc[day]/confirmed.iloc[day-1])*100
    
active_cases=confirmed.copy()
active_rate=confirmed.copy()

for day in range(0,len(confirmed)):
    active_cases.iloc[day]=confirmed.iloc[day]-recovered.iloc[day]
    active_rate.iloc[day]=((confirmed.iloc[day]-recovered.iloc[day]-death.iloc[day])/active_cases.iloc[day-1])*100
    
overall_growth_rate=confirmed.copy()
for day in range(1,len(confirmed)):
    overall_growth_rate.iloc[day]=((active_cases.iloc[day]-active_cases.iloc[day-1])/
                                  ( active_cases.iloc[day-1]))*100
                                
death_rate=confirmed.copy()
for day in range(1,len(confirmed)):
    death_rate.iloc[day]=100*(death.iloc[day]-death.iloc[day-1])/death.iloc[day-1]
 

death_rate=confirmed.copy()
conf_rollavg=confirmed.copy()
recov_rollavg=confirmed.copy()
death_rollavg=confirmed.copy()
active_rollavg=confirmed.copy()
aceleration_conf=confirmed.copy()

for day in range(0,len(confirmed)):
    death_rate.iloc[day]=(death.iloc[day]/confirmed.iloc[day])*100
    conf_rollavg.iloc[day]=confirmed.iloc[day]/day
    recov_rollavg.iloc[day]=recovered.iloc[day]/day
    death_rollavg.iloc[day]=death.iloc[day]/day
    active_rollavg.iloc[day]=confirmed.iloc[day]/day-recovered.iloc[day]/day-death.iloc[day]/day
    

    
# Visualisation
countries=['Italy','Sweden','India','US','France','Spain']


In [ ]:
recovery_rate=confirmed.copy()                               
for day in range(1,len(confirmed)):
    recovery_rate.iloc[day]=((confirmed.iloc[day]-active_cases.iloc[day]-death.iloc[day])/
                                  ( confirmed.iloc[day]))*100

In [ ]:
death_rate_ts=death_rate.India
growth_rate_ts=growth_rate.India
active_rate_ts=active_rate.India
recovery_rate_ts=recovery_rate.India

In [ ]:
plt.plot(death_rate_ts.index,death_rate_ts.values)

In [ ]:
ts_array = [death_rate_ts[100:], growth_rate_ts[100:], active_rate_ts[100:],recovery_rate_ts[100:]]
labels = ['DeathRate', 'GrowthRate', 'ActiveRate', 'RecoveryRate']
colors = [color_arr[5],  color_arr[0], color_arr[2], color_arr[3]]
mode_size = [8, 8, 12, 8]
line_size = [2, 2, 4, 2]

# Calling the draw_plot function defined above
fig_3 = draw_plot(ts_array = ts_array, 
                  ts_label = labels , 
                  title = "Covid-19 Case Trend in INDIA",
                  colors = colors, mode_size = mode_size, 
                  line_size = line_size , 
                  x_axis_title = "Date" , 
                  y_axis_title = "Case Count", 
                  tickangle = 315, 
                  yaxis_type = "", additional_annotations=[])

fig_3.show()

In [ ]:
death_rate_ts

In [ ]:
new_cases.India
plt.figure(figsize=(15,8))
plt.plot(confirmed.index[100:],active_rate.India[100:],label="Active Rate of India")
plt.plot(confirmed.index[100:],overall_growth_rate.India[100:],label="Growth Rate of India")
plt.plot(confirmed.index[100:],recovery_rate.India[100:],label="Recovery Rate of India")
plt.suptitle("Recent Covid 19  Cases in India (upto August 28th 2020)")
plt.xlabel("Days")
plt.ylabel("% Increase or Decrease or Different Cases")
plt.text(10,20,"Anathnath Ghosh",fontsize=65,color='gray',alpha=.4,rotation=25)
plt.legend(loc='upper right')
plt.show()

In [ ]:
new_cases.India

In [ ]:
growth_India_ts=growth_rate.India


In [ ]:
growth_India_ts

In [ ]:
conf_velocity=confirmed.copy()
for day in range(1,len(confirmed)):
     conf_velocity.iloc[day]=(confirmed.iloc[day]-confirmed.iloc[day-1])/day

In [ ]:
conf_ace=confirmed.copy()
for day in range(1,len(confirmed)):
     conf_ace.iloc[day]=(conf_velocity.iloc[day]-conf_velocity.iloc[day-1])/day

In [ ]:
new_cases

# Use of plotly

SIR MODEL STUDY 29th August 2020

In [ ]:
frac_inf=confirmed.India.max()/1350000000
frac_recov=recovered.India.max()/confirmed.India.max()
frac_sus=1-frac_inf

In [ ]:
frac_inf
frac_recov
frac_sus

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
ndays=100
dt=.1 # step in days
npts=int(ndays/dt)
beta=(1.0/3) # Infection rate
gamma=(1.0/14.0) # Recovery rate
S=np.zeros(npts) # Susceptible
I=np.zeros(npts) # Infected
R=np.zeros(npts) # Recovered persons
t=np.arange(npts)*dt
I[0]=frac_inf # Infected proportin
S[0]=1-I[0]
R[0]=frac_recov

for i in range(npts-1):
  S[i+1]=S[i]-beta*(S[i]*I[i])*dt
  I[i+1]=I[i]+(beta*I[i]*S[i]-gamma*I[i])*dt
  R[i+1]=R[i]+(gamma*I[i])*dt

fig=plt.figure(1)
plt.plot(t,S,'r',lw=3,label="Susceptible")
plt.plot(t,I,'g',lw=3,label="Infected")
plt.plot(t,R,'b',lw=3,label="Recovered")
fig.legend(); plt.xlabel("days");plt.ylabel(" Cases")
fig.show()

In [ ]:
import seaborn as  sns
plt.figure(figsize=(20,15))
sns.barplot(x=growth_rate.index,y=growth_rate.India,label='Confirmed',
            color='g')

In [ ]:
import plotly.express as px

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
ndays=100
dt=.1 # step in days
npts=int(ndays/dt)
beta=(1.0/3.0) # Infection rate
gamma=(1.0/14.0) # Recovery rate
S=np.zeros(npts) # Susceptible
I=np.zeros(npts) # Infected
R=np.zeros(npts) # Recovered persons
t=np.arange(npts)*dt
I[0]=frac_inf # Infected proportin
S[0]=1-I[0]
R[0]=frac_recov

for i in range(npts-1):
  S[i+1]=S[i]-beta*(S[i]*I[i])*dt
  I[i+1]=I[i]+(beta*I[i]*S[i]-gamma*I[i])*dt
  R[i+1]=R[i]+(gamma*I[i])*dt

fig=plt.figure(1)
plt.plot(t,S,'r',lw=3,label="Susceptible")
plt.plot(t,I,'g',lw=3,label="Infected")
plt.plot(t,R,'b',lw=3,label="Recovered")
fig.legend(); plt.xlabel("days");plt.ylabel(" Different fractional Cases")
fig.show()

In [ ]:
I.max()

In [ ]:
#@title
x=confirmed.index
y=conf_ace.India
y1=conf_ace.Italy
y2=conf_ace.US
z=conf_velocity.India
z1=conf_velocity.Italy
z2=conf_velocity.US
m=overall_growth_rate.India
m1=overall_growth_rate.Italy
m2=overall_growth_rate.US
s=confirmed.India
s1=confirmed.Italy
s2=confirmed.US
plt.subplots(figsize=(15,10))
plt.subplot(221)
plt.plot(x[90:],y.values[90:],x[90:],y1[90:],x[90:],y2[90:])
plt.title('Data Aceeleration of India. Italy and US')
plt.subplot(222)
plt.plot(x[90:],z.values[90:],x[90:],z1[90:],x[90:],z2[90:])
plt.title('Data velocity of India, Italy and US')
plt.subplot(223)
plt.plot(x[90:],m.values[90:],x[90:],m1[90:],x[90:],m2[90:])
plt.title('Overall growth rate of India, Italy and US')
plt.subplot(224)
plt.plot(x[90:],s.values[90:],x[90:],s1[90:],x[90:],s2[90:])
plt.title('Confirmed Cases')
plt.show()
#y=aceleration_conf.India
#y.values
#y.index



In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

output_file("layout_grid.html")
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_ace.India
q=conf_ace.Brazil
z=conf_ace.US
m=conf_ace.Italy

y1=conf_velocity.India
q1=conf_velocity.Brazil
m1=conf_velocity.Italy
z1=conf_velocity.US

y2=overall_growth_rate.India
q2=overall_growth_rate.Brazil
m2=overall_growth_rate.Italy
z2=overall_growth_rate.US

y3=confirmed.India
q3=confirmed.Brazil
m3=confirmed.Italy
z3=confirmed.US

# create three plots
s1 = figure(background_fill_color="#fafafa")
s1.circle(x, y.values, size=1, alpha=0.8, legend_label='India-ace',color='orange')
s1.square(x, z.values, size=2, alpha=0.3, legend_label='Brazil-ace')
s1.triangle(x, m.values, size=3, alpha=0.93, legend_label='Italy-ace',color='green')
s1.square_cross(x, q.values, size=4, alpha=0.93, legend_label='US-ace',color='red')

s2 = figure(background_fill_color="#fafafa")
s2.triangle(x, y1.values, size=1, alpha=0.8, legend_label='India-velocity',color='orange')
s2.triangle(x, z1.values, size=1, alpha=0.3, legend_label='Brazil-velocity')
s2.triangle(x, m1.values, size=1, alpha=0.93, legend_label='Italy-velocity',color='green')
s2.triangle(x, q1.values, size=1, alpha=0.93, legend_label='US-velocity',color='red')

s3 = figure(background_fill_color="#fafafa")
s3.square(x, y2.values, size=1, alpha=0.8, legend_label='India-growthrate',color='orange')
s3.square(x, z2.values, size=1, alpha=0.3, legend_label='Brazil-growthrate')
s3.square(x, m2.values, size=1, alpha=0.93, legend_label='Italy-growthrate',color='green')
s3.square(x, q2.values, size=1, alpha=0.93, legend_label='US-growthrate',color='red')

s4 = figure(background_fill_color="#fafafa")
s4.square(x, y3.values, size=1, alpha=0.8, legend_label='India-confirmed',color='orange')
s4.square(x, z3.values, size=1, alpha=0.3, legend_label='Brazil-confirmed')
s4.square(x, m3.values, size=1, alpha=0.93, legend_label='Italy-confirmed',color='green')
s4.square(x, q3.values, size=1, alpha=0.93, legend_label='US-confirmed',color='red')

# make a grid
grid = gridplot([[s1, s2], [s3, s4]], plot_width=250, plot_height=250)
show(grid)


In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

output_file("layout_grid.html")
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_ace.India
q=conf_ace.Brazil
z=conf_ace.US
m=conf_ace.Italy

y1=conf_velocity.India
q1=conf_velocity.Brazil
m1=conf_velocity.Italy
z1=conf_velocity.US

y2=overall_growth_rate.India
q2=overall_growth_rate.Brazil
m2=overall_growth_rate.Italy
z2=overall_growth_rate.US

y3=confirmed.India
q3=confirmed.Brazil
m3=confirmed.Italy
z3=confirmed.US

# create three plots
s1 = figure(background_fill_color="#fafafa")
s1.circle(x, y.values, size=1, alpha=0.99, legend_label='India-ace',color='red')

s2 = figure(background_fill_color="#fafafa")
s2.triangle(x, y1.values, size=1, alpha=0.8, legend_label='India-velocity',color='orange')

s3 = figure(background_fill_color="#fafafa")
s3.square(x, y2.values, size=1, alpha=0.8, legend_label='India-growthrate',color='orange')

s4 = figure(background_fill_color="#fafafa")
s4.square(x, y3.values, size=1, alpha=0.8, legend_label='India-confirmed',color='orange')

# make a grid
grid = gridplot([[s1, s2], [s3, s4]], plot_width=250, plot_height=250)
show(grid)


In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

output_file("layout_grid.html")
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_ace.India
q=conf_ace.Brazil
z=conf_ace.US
m=conf_ace.Italy

y1=conf_velocity.India
q1=conf_velocity.Brazil
m1=conf_velocity.Italy
z1=conf_velocity.US

y2=overall_growth_rate.India
q2=overall_growth_rate.Brazil
m2=overall_growth_rate.Italy
z2=overall_growth_rate.US

y3=confirmed.India
q3=confirmed.Brazil
m3=confirmed.Italy
z3=confirmed.US

# create three plots
s1 = figure(background_fill_color="#fafafa")
s1.triangle(x, m.values, size=3, alpha=0.93, legend_label='Italy-ace',color='green')

s2 = figure(background_fill_color="#fafafa")
s2.triangle(x, m1.values, size=1, alpha=0.93, legend_label='Italy-velocity',color='green')

s3 = figure(background_fill_color="#fafafa")
s3.square(x, m2.values, size=1, alpha=0.93, legend_label='Italy-growthrate',color='green')

s4 = figure(background_fill_color="#fafafa")
s4.square(x, m3.values, size=1, alpha=0.93, legend_label='Italy-confirmed',color='green')

# make a grid
grid = gridplot([[s1, s2], [s3, s4]], plot_width=250, plot_height=250)
show(grid)


In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

output_file("layout_grid.html")
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_ace.India
q=conf_ace.Brazil
z=conf_ace.US
m=conf_ace.Italy

y1=conf_velocity.India
q1=conf_velocity.Brazil
m1=conf_velocity.Italy
z1=conf_velocity.US

y2=overall_growth_rate.India
q2=overall_growth_rate.Brazil
m2=overall_growth_rate.Italy
z2=overall_growth_rate.US

y3=confirmed.India
q3=confirmed.Brazil
m3=confirmed.Italy
z3=confirmed.US

# create three plots
s1 = figure(background_fill_color="#fafafa")
s1.square(x, z.values, size=2, alpha=0.99, legend_label='Brazil-ace',color='green')

s2 = figure(background_fill_color="#fafafa")
s2.triangle(x, z1.values, size=1, alpha=0.99, legend_label='Brazil-velocity',color='red')

s3 = figure(background_fill_color="#fafafa")
s3.square(x, z2.values, size=1, alpha=0.99, legend_label='Brazil-growthrate',color='yellow')

s4 = figure(background_fill_color="#fafafa")
s4.square(x, z3.values, size=1, alpha=0.99, legend_label='Brazil-confirmed',color='orange')

# make a grid
grid = gridplot([[s1, s2], [s3, s4]], plot_width=250, plot_height=250)
show(grid)


In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

output_file("layout_grid.html")
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_ace.India
q=conf_ace.Brazil
z=conf_ace.US
m=conf_ace.Italy

y1=conf_velocity.India
q1=conf_velocity.Brazil
m1=conf_velocity.Italy
z1=conf_velocity.US

y2=overall_growth_rate.India
q2=overall_growth_rate.Brazil
m2=overall_growth_rate.Italy
z2=overall_growth_rate.US

y3=confirmed.India
q3=confirmed.Brazil
m3=confirmed.Italy
z3=confirmed.US

# create three plots
s1 = figure(background_fill_color="#fafafa")
s1.square_cross(x, q.values, size=4, alpha=0.93, legend_label='US-aceleration',color='red')

s2 = figure(background_fill_color="#fafafa")
s2.triangle(x, q1.values, size=1, alpha=0.93, legend_label='US-velocity',color='red')

s3 = figure(background_fill_color="#fafafa")
s3.square(x, q2.values, size=1, alpha=0.93, legend_label='US-growthrate',color='red')

s4 = figure(background_fill_color="#fafafa")
s4.square(x, q3.values, size=1, alpha=0.93, legend_label='US-confirmed',color='red')

# make a grid
grid = gridplot([[s1, s2], [s3, s4]], plot_width=250, plot_height=250)
show(grid)


In [ ]:
ace_roll_avg=confirmed.copy()
velocity_roll_avg=confirmed.copy()
for i in range(1,len(confirmed)):
     ace_roll_avg.iloc[day]=(conf_ace.iloc[day])/day
     velocity_roll_avg.iloc[day]=(conf_velocity.iloc[day])/day

In [ ]:
d=ace_roll_avg.India
d1=ace_roll_avg.Brazil
d2=ace_roll_avg.Italy
d3=ace_roll_avg.US



plt.plot(x,d.values,x,d1.values,x,d2.values,x,d3.values)

In [ ]:
plt.figure(figsize=(12,8))
countrywise.plot(kind='bar',x='Country_Region',y='Confirmed')
plt.show()

In [ ]:
import plotly.express as px
px.bar(countrywise,x='Country_Region',y='Confirmed')

In [ ]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
x=countrywise.index
y=countrywise.Confirmed
y1=countrywise.Recovered
y2=countrywise.Deaths
p=figure(plot_width=500,plot_height=400)
p.outline_line_width=7
p.outline_line_alpha=.3
p.outline_line_color='navy'
p.title.text_color='olive'
p.title.text_font='times'
p.title.text_font_style='italic'
p.background_fill_color='wheat'
p.line(x,y,legend_label="Confirmed",line_color='green')
p.line(x,y1,legend_label="Recovered",line_dash=[4,4],line_color='red')
p.line(x,y2,legend_label="Recovered",line_dash=[4,4],line_color='black')

In [ ]:
show(p)

In [ ]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_rollavg.US
y1=recov_rollavg.US
y2=death_rollavg.US
p=figure(plot_width=1000,plot_height=500)
p.outline_line_width=7
p.outline_line_alpha=.3
p.outline_line_color='navy'
p.title.text_color='olive'
p.title.text_font='times'
p.title.text_font_style='italic'
p.background_fill_color='wheat'
p.line(x,y.values,legend_label="Confirmed",line_color='green')
p.line(x,y1.values,legend_label="Recovered",line_dash=[4,4],line_color='red')
p.line(x,y2.values,legend_label="Death",line_dash=[4,4],line_color='black')

In [ ]:
show(p)

In [ ]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_rollavg.India
y1=recov_rollavg.India
y2=death_rollavg.India
p=figure(plot_width=1000,plot_height=500)
p.outline_line_width=7
p.outline_line_alpha=.3
p.outline_line_color='navy'
p.title.text_color='olive'
p.title.text_font='times'
p.title.text_font_style='italic'
p.background_fill_color='wheat'
p.line(x,y.values,legend_label="Confirmed",line_color='green')
p.line(x,y1.values,legend_label="Recovered",line_dash=[4,4],line_color='red')
p.line(x,y2.values,legend_label="Death",line_dash=[4,4],line_color='black')

In [ ]:
show(p)

In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import row
from bokeh.plotting import figure

output_file("layout.html")
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_rollavg.India
y1=recov_rollavg.India
y2=death_rollavg.India
#x = list(range(11))
#y0 = x
#y1 = [10 - i for i in x]
#y2 = [abs(i - 5) for i in x]

# create three plots
s1 = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa")
s1.circle(x, y.values, size=1,legend_label='Confirmed-India', alpha=0.8)

s2 = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa")
s2.triangle(x, y1.values, size=1, legend_label='Recovered-India', alpha=0.8)

s3 = figure(plot_width=250, plot_height=250, background_fill_color="#fafafa")
s3.square(x, y2.values, size=1, color="#d95b43", alpha=0.8)

# put the results in a column and show
#show(column(s1, s2, s3))
show(row(s1, s2, s3))


In [ ]:
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure

output_file("layout_grid.html")
x=np.linspace(1,len(confirmed),len(confirmed.index))
y=conf_rollavg.India
q=conf_rollavg.Brazil
z=conf_rollavg.US
m=conf_rollavg.Italy
y1=recov_rollavg.India
q1=recov_rollavg.Brazil
m1=recov_rollavg.Italy
z1=recov_rollavg.US
y2=death_rollavg.India
q2=death_rollavg.Brazil
m2=death_rollavg.Italy
z2=death_rollavg.US
y3=active_rollavg.India
q3=active_rollavg.Brazil
m3=active_rollavg.Italy
z3=active_rollavg.US

# create three plots
s1 = figure(background_fill_color="#fafafa")
s1.circle(x, y.values, size=1, alpha=0.8, legend_label='India-confirmed',color='orange')
s1.circle(x, z.values, size=1, alpha=0.3, legend_label='US-confirmed')
s1.circle(x, m.values, size=1, alpha=0.93, legend_label='Italy-confirmed',color='green')
s1.circle(x, q.values, size=1, alpha=0.93, legend_label='Brazil-confirmed',color='red')

s2 = figure(background_fill_color="#fafafa")
s2.triangle(x, y1.values, size=1, alpha=0.8, legend_label='India-recovered',color='orange')
s2.triangle(x, z1.values, size=1, alpha=0.3, legend_label='US-recovered')
s2.triangle(x, m1.values, size=1, alpha=0.93, legend_label='Italy-recovered',color='green')
s2.triangle(x, q1.values, size=1, alpha=0.93, legend_label='Brazil-recovered',color='red')

s3 = figure(background_fill_color="#fafafa")
s3.square(x, y2.values, size=1, alpha=0.8, legend_label='India-death',color='orange')
s3.square(x, z2.values, size=1, alpha=0.3, legend_label='US-death')
s3.square(x, m2.values, size=1, alpha=0.93, legend_label='Italy-death',color='green')
s3.square(x, q2.values, size=1, alpha=0.93, legend_label='Brazil-death',color='red')

s4 = figure(background_fill_color="#fafafa")
s4.square(x, y3.values, size=1, alpha=0.8, legend_label='India-active',color='orange')
s4.square(x, z3.values, size=1, alpha=0.3, legend_label='US-active')
s4.square(x, m3.values, size=1, alpha=0.93, legend_label='Italy-active',color='green')
s4.square(x, q3.values, size=1, alpha=0.93, legend_label='Brazil-active',color='red')

# make a grid
grid = gridplot([[s1, s2], [s3, s4]], plot_width=250, plot_height=250)

show(grid)

In [ ]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=conf_rollavg.US
y1=conf_rollavg.India
y2=conf_rollavg.Brazil
y3=conf_rollavg.Italy
p=figure(plot_width=1000,plot_height=500)
p.outline_line_width=7
p.outline_line_alpha=.3
p.outline_line_color='navy'
p.title.text_color='olive'
p.title.text_font='times'
p.title.text_font_style='italic'
p.background_fill_color='wheat'
p.line(x,y.values,legend_label="Confirmed-US",line_color='green')
p.line(x,y1.values,legend_label="Confirmed-India",line_dash=[4,4],line_color='red')
p.line(x,y2.values,legend_label="Confirmed-Brazil",line_dash=[4,4],line_color='black')
p.line(x,y3.values,legend_label="Confirmed-Italy",line_dash=[4,4],line_color='orange')

In [ ]:
show(p)

In [ ]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=recov_rollavg.US
y1=recov_rollavg.India
y2=recov_rollavg.Brazil
y3=recov_rollavg.Italy
p=figure(plot_width=1000,plot_height=500)
p.outline_line_width=7
p.outline_line_alpha=.3
p.outline_line_color='navy'
p.title.text_color='olive'
p.title.text_font='times'
p.title.text_font_style='italic'
p.background_fill_color='wheat'
p.line(x,y.values,legend_label="Recovered-US",line_color='green')
p.line(x,y1.values,legend_label="Recovered-India",line_dash=[4,4],line_color='red')
p.line(x,y2.values,legend_label="Recovered-Brazil",line_dash=[4,4],line_color='black')
p.line(x,y3.values,legend_label="Recovered-Italy",line_dash=[4,4],line_color='orange')

In [ ]:
show(p)

In [ ]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=death_rollavg.US
y1=death_rollavg.India
y2=death_rollavg.Brazil
y3=death_rollavg.Italy
p=figure(plot_width=1000,plot_height=500)
p.outline_line_width=7
p.outline_line_alpha=.3
p.outline_line_color='navy'
p.title.text_color='olive'
p.title.text_font='times'
p.title.text_font_style='italic'
p.background_fill_color='white'
p.line(x,y.values,legend_label="Death-US",line_color='green')
p.line(x,y1.values,legend_label="Death-India",line_dash=[4,4],line_color='red')
p.line(x,y2.values,legend_label="Death-Brazil",line_dash=[4,4],line_color='black')
p.line(x,y3.values,legend_label="Death-Italy",line_dash=[4,4],line_color='orange')

In [ ]:
show(p)

In [ ]:
import bokeh
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(resources=INLINE)
x=np.linspace(1,len(confirmed.index),len(confirmed.index))
y=active_rollavg.US
y1=active_rollavg.India
y2=active_rollavg.Brazil
y3=active_rollavg.Italy
p=figure(plot_width=1000,plot_height=500)
p.outline_line_width=7
p.outline_line_alpha=.3
p.outline_line_color='navy'
p.title.text_color='olive'
p.title.text_font='times'
p.title.text_font_style='italic'
p.background_fill_color='white'
p.line(x,y.values,legend_label="Active-US",line_color='green')
p.line(x,y1.values,legend_label="Active-India",line_dash=[4,4],line_color='red')
p.line(x,y2.values,legend_label="Active-Brazil",line_dash=[4,4],line_color='black')
p.line(x,y3.values,legend_label="Active-Italy",line_dash=[4,4],line_color='orange')

In [ ]:
show(p)

In [ ]:
plt.plot(y.index,y.values)

In [ ]:
len(y.index)

In [ ]:
len(y.values)

In [ ]:
show(p)

In [ ]:
import numpy as np
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# output to static HTML file
output_file("linked_panning.html")

# create a new plot
s1 = figure(width=250, plot_height=250, title=None)
s1.circle(x, y0, size=2, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(width=250, height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=3, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(width=250, height=250, x_range=s1.x_range, title=None)
s3.square(x, y2, size=4, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None)

# show the results
show(p)

In [ ]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# output to static HTML file
output_file("linked_panning.html")

# create a new plot
s1 = figure(width=250, plot_height=250, title=None)
s1.circle(x, y0, size=10, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(width=250, height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=10, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(width=250, height=250, x_range=s1.x_range, title=None)
s3.square(x, y2, size=10, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None)

# show the results
show(p)

In [ ]:
show(p)

In [ ]:
len(y)

In [ ]:
show(p)


In [ ]:
show(p)

In [ ]:
import seaborn as  sns
plt.figure(figsize=(20,15))
sns.barplot(x=countrywise.index,y=countrywise.Confirmed,data=countrywise,label='Confirmed',
            color='g')

In [ ]:
import plotly.express as px
fig=px.bar(countrywise,x='Country_Region',y='Confirmed',barmode='group',height=500)

In [ ]:
fig.show()

In [ ]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=countrywise['Country_Region'],y=countrywise['Confirmed'],
                         mode='lines+markers',name='Confirmed Cases'))


In [ ]:
for country in countries:
    ax=plt.subplot()
    ax.set_facecolor('black')
    ax.figure.set_facecolor('#121212')
    ax.tick_params(axis='x',color='white')
    ax.tick_params(axis='y',color='white')
    ax.set_title(r'sumulted growth rate  by {country}')
#for country in countries:
    confirmed[country][40:].plot(figsize=(10,10),label=country)
    recovered[country][40:].plot(figsize=(10,10),label=country)
    death[country][40:].plot(figsize=(10,10),label=country)
    plt.legend(loc='upper left')
    plt.show()    

In [ ]:
for country in countries:
    #growth_rate[country].plot(figsize=(10,10),label=country)
    conf_rollavg[country].plot(figsize=(10,10),label=country)
    recov_rollavg[country].plot(figsize=(10,10),label=country)
    death_rollavg[country].plot(figsize=(10,10),label=country)
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
for country in countries:
     death_rate[country].plot.bar()
     plt.legend(loc='upper left')
     plt.show()

In [ ]:
import plotly.graph_objs as go
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
plt.subplot(221)
x=conf_rollavg.index
y=conf_rollavg.US
fig=go.FigureWidget(data=[{'x':x,'y':y,'mode':'markers'}])
plt.subplot(222)
x=conf_rollavg.index
y=conf_rollavg.India
fig=go.FigureWidget(data=[{'x':x,'y':y,'mode':'markers'}])
fig.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(221)
plt.plot(x,conf_rollavg.US,x,conf_rollavg.India,x,conf_rollavg.Italy,x,conf_rollavg.Brazil)
plt.subplot(222)
plt.plot(x,recov_rollavg.US,x,recov_rollavg.India,x,recov_rollavg.Italy,x,recov_rollavg.Brazil)
plt.subplot(223)
plt.plot(x,death_rollavg.US,x,death_rollavg.India,x,death_rollavg.Italy,x,death_rollavg.Brazil)
plt.subplot(224)
plt.plot(x,active_rollavg.US,x,active_rollavg.India,x,active_rollavg.Italy,x,active_rollavg.Brazil)


In [ ]:
plt.figure(figsize = (20, 10))
plt.subplot(221)
plt.plot(x, conf_rollavg.US, '-o', x,death_rollavg.US, '-o',x,recov_rollavg.US, '-o')
plt.text(50, 10000, 'US', size = 18)
#plt.xlim(40, 65)
plt.legend(['Confirmed', 'Death','Recovery'])
plt.xlabel('Days', size = 16)
plt.subplot(222)
plt.plot(x, conf_rollavg.India, '-o', x, death_rollavg.India, '-o',x,recov_rollavg.US, '-o')
plt.text(55, 10000, 'India', size = 18)
#plt.xlim(20, 65)
plt.legend(['Confirmed', 'Death','Recovery'])
plt.xlabel('Days', size = 16)
plt.subplot(223)
plt.plot(x, conf_rollavg.Brazil, '-o', x, death_rollavg.Brazil, '-o',x,recov_rollavg.US, '-o')
plt.text(55, 10000, 'Brazil', size = 18)
#plt.xlim(30, 65)
plt.legend(['Confirmed', 'Death','Recovery'])
plt.subplot(224)
plt.plot(x, conf_rollavg.Italy, '-o', x, death_rollavg.Italy, '-o',x,recov_rollavg.US, '-o')
plt.text(55, 1000, 'Italy', size = 18)
#plt.xlim(30, 65)
plt.legend(['Confirmed', 'Death','Recovery'])
plt.show()

COMPARING AMONG DIFFERENT COUNTRIES

In [ ]:
plt.figure(figsize = (20, 10))
plt.subplot(221)
plt.plot(x, conf_rollavg.US, '-o', x,conf_rollavg.India, '-o',
         x,conf_rollavg.Brazil, '-o',x,conf_rollavg.Italy, '-o')
plt.text(50, 10000, 'CONFIRMED', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(222)
plt.plot(x, recov_rollavg.US, '-o', x,recov_rollavg.India, '-o',
         x,recov_rollavg.Brazil, '-o',x,recov_rollavg.Italy, '-o')
plt.text(50, 10000, 'RECOVERED', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(223)
plt.plot(x, death_rollavg.US, '-o', x,death_rollavg.India, '-o',
         x,death_rollavg.Brazil, '-o',x,death_rollavg.Italy, '-o')
plt.text(50, 100, 'Death', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(224)
plt.plot(x, active_rollavg.US, '-o', x,active_rollavg.India, '-o',
         x,active_rollavg.Brazil, '-o',x,active_rollavg.Italy, '-o')
plt.text(50, 1000, 'ACTIVE', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
plt.subplot(221)
plt.semilogy(x, conf_rollavg.US, '-o', x,conf_rollavg.India, '-o',
         x,conf_rollavg.Brazil, '-o',x,conf_rollavg.Italy, '-o')
plt.text(50, 10000, 'CONFIRMED', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(222)
plt.semilogy(x, recov_rollavg.US, '-o', x,recov_rollavg.India, '-o',
         x,recov_rollavg.Brazil, '-o',x,recov_rollavg.Italy, '-o')
plt.text(50, 10000, 'RECOVERED', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(223)
plt.semilogy(x, death_rollavg.US, '-o', x,death_rollavg.India, '-o',
         x,death_rollavg.Brazil, '-o',x,death_rollavg.Italy, '-o')
plt.text(50, 100, 'Death', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(224)
plt.semilogy(x, active_rollavg.US, '-o', x,active_rollavg.India, '-o',
         x,active_rollavg.Brazil, '-o',x,active_rollavg.Italy, '-o')
plt.text(50, 1000, 'ACTIVE', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
plt.subplot(221)
plt.loglog(x, conf_rollavg.US, '-o', x,conf_rollavg.India, '-o',
         x,conf_rollavg.Brazil, '-o',x,conf_rollavg.Italy, '-o')
plt.text(50, 10000, 'CONFIRMED', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(222)
plt.loglog(x, recov_rollavg.US, '-o', x,recov_rollavg.India, '-o',
         x,recov_rollavg.Brazil, '-o',x,recov_rollavg.Italy, '-o')
plt.text(50, 10000, 'RECOVERED', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(223)
plt.loglog(x, death_rollavg.US, '-o', x,death_rollavg.India, '-o',
         x,death_rollavg.Brazil, '-o',x,death_rollavg.Italy, '-o')
plt.text(50, 100, 'Death', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.subplot(224)
plt.loglog(x, active_rollavg.US, '-o', x,active_rollavg.India, '-o',
         x,active_rollavg.Brazil, '-o',x,active_rollavg.Italy, '-o')
plt.text(50, 1000, 'ACTIVE', size = 18)
#plt.xlim(40, 65)
plt.legend(['US', 'INDIA','Brazil','Italy'])
plt.xlabel('Days', size = 16)
plt.show()

In [ ]:
simulated_growth_rate =.10
dates=pd.date_range(start='06/11/2010',periods=40,freq='D')
dates=pd.Series(dates)
dates=dates.dt.strftime('%m/%d/%Y')
simulated=confirmed.copy()
simulated=simulated.append(pd.DataFrame(index=dates))

for day in range(len(confirmed),len(confirmed)+40):
     simulated.iloc[day]=(simulated.iloc[day-1])*(1+simulated_growth_rate)

for country in countries:
     simulated[country].plot(figsize=(10,10),label=country)
     plt.legend(loc='upper left')
     plt.show()

In [ ]:
data_conf = pd.read_csv(confirmedlink)
data_recov = pd.read_csv(recoveredlink)
data_death = pd.read_csv(deathlink)

L_conf = data_conf.values.tolist()
L_recov = data_recov.values.tolist()
L_death = data_death.values.tolist()
data_conf.loc[data_conf['Country/Region'] == 'India']
conf_india = L_conf[131] # INDIA
recov_india = L_recov[125]
death_india = L_death[131]
conf_italy = L_conf[137] # ITALY
recov_italy = L_recov[131]
death_italy = L_death[137]
conf_spain = L_conf[201] # Spain
recov_spain = L_recov[199]
death_spain = L_death[201]
conf_us = L_conf[225] # UK
recov_us = L_recov[225]
death_us = L_death[225]
days = range(len(conf_india))

In [ ]:
plt.figure(figsize = (10, 8))
x = np.array(days[6:])
plt.subplot(221)
y = np.diff(np.array(conf_india[5:]))
plt.plot(x, y, '-o')
plt.xlabel('Days', size = 16)
plt.subplot(222)
y = np.diff(np.array(conf_italy[5:]))
plt.plot(x, y, '-o')
plt.xlabel('Days', size = 16)
plt.subplot(223)
y = np.diff(np.array(conf_spain[5:]))
plt.plot(x, y, '-o')
plt.subplot(224)
y = np.diff(np.array(conf_us[5:]))
plt.plot(x, y, '-o')
plt.show()

In [ ]:
f_recov_india = np.array(recov_india[12:])/np.array(conf_india[12:], dtype = float)*100
f_recov_italy = np.array(recov_italy[12:])/np.array(conf_italy[12:], dtype = float)*100
f_recov_spain = np.array(recov_spain[12:])/np.array(conf_spain[12:], dtype = float)*100
f_recov_us = np.array(recov_us[12:])/np.array(conf_us[12:], dtype = float)*100
f_death_india = np.array(death_india[12:])/np.array(conf_india[12:], dtype = float)*100
f_death_italy = np.array(death_italy[12:])/np.array(conf_italy[12:], dtype = float)*100
f_death_spain = np.array(death_spain[12:])/np.array(conf_spain[12:], dtype = float)*100
f_death_us = np.array(death_us[12:])/np.array(death_us[12:],dtype=float)*100


In [ ]:
plt.figure(figsize = (10, 8))
plt.subplot(221)
plt.plot(days[12:], f_death_india, '-o', days[12:], f_recov_india, '-o')
plt.text(50, 80, 'INDIA', size = 18)
plt.xlim(40, 65)
plt.legend(['Death', 'Recovery'])
plt.xlabel('Days', size = 16)
plt.subplot(222)
plt.plot(days[12:], f_death_italy, '-o', days[12:], f_recov_italy, '-o')
plt.text(55, 2, 'ITALY', size = 18)
plt.xlim(20, 65)
plt.legend(['Death', 'Recovery'])
plt.xlabel('Days', size = 16)
plt.subplot(223)
plt.plot(days[12:], f_death_spain, '-o', days[12:], f_recov_spain, '-o')
plt.text(55, 50, 'SPAIN', size = 18)
plt.xlim(30, 65)
plt.legend(['Death', 'Recovery'])
plt.subplot(224)
plt.plot(days[12:], f_death_us, '-o', days[12:], f_recov_us, '-o')
plt.text(55, 60, 'US', size = 18)
plt.xlim(30, 65)
plt.legend(['Death', 'Recovery'])
plt.show()

In [ ]:
data_conf.loc[data_conf['Country/Region'] == 'China']

plt.figure(figsize = (12, 8))
for i in range(49, 81):
    conf_china = L_conf[i]
    plt.semilogy(days[5:], conf_china[5:], lw = 3)
    plt.text(30, 25000, 'Wuhan(Hubei)', size = 18)
    plt.title('CHINA: COVID-19 Confirmed Cases', size = 22)
    plt.xlabel('Days', size = 18)
    plt.tick_params(labelsize = 18)

plt.figure(figsize = (12, 8))
conf_china = L_conf[62]
plt.semilogy(days[5:], conf_china[5:], '-o', lw = 3)
plt.text(40, 20000, 'Wuhan(Hubei)', size = 18)
plt.title('CHINA: COVID-19', size = 26)
plt.xlabel('Days', size = 22)
plt.ylabel('Confirmed Cases', size = 22)
plt.tick_params(labelsize = 18)
plt.show()

for i in range(49, 81):
    conf_china = L_conf[i]
    plt.semilogy(days[5:], conf_china[5:], lw = 3)
    
for i in range(49,81):
    conf_china = L_conf[i]
    plt.semilogy(days[5:], conf_china[5:], lw = 3)
for i in range(49,81):
    conf_china = L_conf[i]
    plt.semilogy(days[5:], conf_china[5:], lw = 3)


In [ ]:
data=data_conf.groupby('Country/Region').sum()
data_transposed=data.T
data_transposed.plot(y=['India','US','Brazil','Italy'],use_index=True,figsize=(15,10),
                     marker='o')

In [ ]:
L_conf = data_conf.values.tolist()
L_recov = data_recov.values.tolist()
L_death = data_death.values.tolist()

In [ ]:
recov_india = L_recov[125]
death_india = L_death[131]

In [ ]:
data_conf.loc[data_conf['Country/Region'] == 'India']
data_death.loc[data_death['Country/Region'] == 'India']
data_recov.loc[data_recov['Country/Region'] == 'India']
conf_india = L_conf[131] # INDIA
recov_india = L_recov[125]
death_india = L_death[131]

In [ ]:
data_conf.loc[data_conf['Country/Region'] == 'India']

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
confimedcaseslink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deathcaseslink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recoveredcaseslink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
countrycaseslink = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"

In [ ]:
confirmed_df = pd.read_csv(confimedcaseslink)
print(confirmed_df.shape)
deaths_df = pd.read_csv(deathcaseslink)
print(deaths_df.shape)
recovered_df = pd.read_csv(recoveredcaseslink)
print(recovered_df.shape)
cases_country_df = pd.read_csv(countrycaseslink)
print(cases_country_df.shape)

In [ ]:
confirmed_df = confirmed_df.replace(np.nan, '', regex=True)
deaths_df = deaths_df.replace(np.nan, '', regex=True)
recovered_df = recovered_df.replace(np.nan, '', regex=True)
cases_country_df = cases_country_df.replace(np.nan, '', regex=True)

In [ ]:
global_data = cases_country_df.copy().drop(['Lat','Long_','Country_Region','Last_Update'],axis =1)
global_summary = pd.DataFrame(global_data.sum()).transpose()
global_summary.style.format(':,.0f')

In [ ]:
confirmed_ts = confirmed_df.copy().drop(['Lat','Long','Country/Region','Province/State'],axis =1)
confirmed_ts_summary = confirmed_ts.sum()

In [ ]:
fig_1 = go.Figure(data=go.Scatter(x=confirmed_ts_summary.index, y = confirmed_ts_summary.values, mode='lines+markers'))
fig_1.update_layout(title='Total Coronavirus Confirmed Cases (Globally)',
                  yaxis_title='Confirmed Cases', xaxis_tickangle = 315 )
fig_1.show()

INDIAN RECENT CASES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
%matplotlib inline
import plotly
import plotly.express as px
import plotly.graph_objects as go
plt.rcParams['figure.figsize']=17,8
import cufflinks as cf
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,plot,iplot
import folium

In [ ]:
df=pd.read_excel('/content/coronaindia7thaugust.xlsx')

In [ ]:
df

In [ ]:
df.drop(['S.No.'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
total_cases_overall=df['Totalactive'].sum()
print('The total number of cases till now in India is ',total_cases_overall)

In [ ]:
df.style.background_gradient(cmap='Reds')

In [ ]:
Total_Active_Cases=df.groupby('Name of State/UT')['Totalactive'].sum().sort_values(ascending=False).to_frame()

In [ ]:
Total_Active_Cases

In [ ]:
Total_Active_Cases.style.background_gradient(cmap='Reds')

In [ ]:
!pip install chart_studio

In [ ]:
#Pandas vis
plt.figure(figsize=(12,10))
df.plot(kind='bar',x='Name of State/UT',y='Totalactive')
plt.show()



In [ ]:
#plotly
#df.iplot(kind='bar',x='Name of State/UT',y='Totalactive')

In [ ]:
px.bar(df,x='Name of State/UT',y='Totalactive')

In [ ]:
df.plot(kind='scatter',x='Name of State/UT',y='Totalactive')

In [ ]:
plt.scatter(df['Name of State/UT'],df['Totalactive'])

In [ ]:
#df.iplot(kind='scatter',x='Name of State / UT',y='Totalactive',mode='markers+lines',title='My Graph',xTitle='Name of State / UT',yTitle='Total Cases',colors='red',size=20)
px.scatter(df,x='Name of State/UT',y='Totalactive')

In [ ]:
#Matplotlib
fig=plt.figure(figsize=(20,10),dpi=200)
axes=fig.add_axes([0,0,1,1])
axes.bar(df['Name of State/UT'],df['Totalactive'])
axes.set_title("Total Cases in India")
axes.set_xlabel("Name of State/UT")
axes.set_ylabel("Totalactive")
plt.show()


In [ ]:
#plotly
fig=go.Figure()
fig.add_trace(go.Bar(x=df['Name of State/UT'],y=df['Totalactive']))
fig.update_layout(title='Total Cases in India',xaxis=dict(title='Name of State/UT'),yaxis=dict(title='Total Cases'))

In [ ]:
#plotly
fig=go.Figure()
fig.add_trace(go.Bar(x=df['Name of State/UT'],y=df['Totalactive']))
fig.update_layout(title='Total Cases in India',xaxis=dict(title='Name of State/UT'),yaxis=dict(title='Total Cases'))

In [ ]:
Indian_Cord=pd.read_excel('/content/Indian Coordinates.xlsx')


In [ ]:
Indian_Cord

In [ ]:
#df_full=pd.merge(Indian_Cord,df,on='Name of State/UT')
#Points=Indian_Cord.apply(lambda row: row.Latitude,row.Longitude)


In [ ]:
map=folium.Map(location=[20,70],zoom_start=4,tiles='Stamenterrain')

for lat,long,value, name in zip(Indian_Cord['Latitude'],Indian_Cord['Longitude'],df['Totalactive'],df['Name of State/UT']):
    folium.CircleMarker([lat,long],radius=value*0.8,popup=('<strong>State</strong>: '+str(name).capitalize()+'<br>''<strong>Total Cases</strong>: ' + str(value)+ '<br>'),color='red',fill_color='red',fill_opacity=0.3).add_to(map)
    

In [ ]:
dbd_India=pd.read_excel(r'/content/per_day_cases.xlsx',parse_dates=True,sheet_name="India")
dbd_Italy=pd.read_excel(r'/content/per_day_cases.xlsx',parse_dates=True,sheet_name="Italy")
dbd_Korea=pd.read_excel(r"/content/per_day_cases.xlsx",parse_dates=True,sheet_name="Korea")
dbd_Wuhan=pd.read_excel(r"/content/per_day_cases.xlsx",parse_dates=True,sheet_name="Wuhan")

In [ ]:
dbd_India

In [ ]:
#Matplotlib
fig=plt.figure(figsize=(10,5),dpi=200)
axes=fig.add_axes([0.1,0.1,0.8,0.8])
axes.bar(dbd_India["Date"],dbd_India["Total Cases"],color='blue')
axes.set_xlabel("Date")
axes.set_ylabel("Total Cases")
axes.set_title("Confirmed cases in India")
plt.show()

In [ ]:
confirmed_India_ts = confirmed_df[confirmed_df['Country/Region']=="India"]
confirmed_India_ts = confirmed_India_ts.drop(
    ['Lat','Long','Country/Region','Province/State'],axis =1).reset_index(drop=True).sum()

deaths_India_ts = deaths_df[deaths_df['Country/Region']=="India"]
deaths_India_ts = deaths_India_ts.drop(
    ['Lat','Long','Country/Region','Province/State'],axis =1).reset_index(drop=True).sum()

recovered_India_ts = recovered_df[recovered_df['Country/Region']=="India"]
recovered_India_ts = recovered_India_ts.drop(
    ['Lat','Long','Country/Region','Province/State'],axis =1).reset_index(drop=True).sum()

active_India_ts = pd.Series(
    data=np.array(
        [x1 - x2 - x3  for (x1, x2, x3) in zip(
            confirmed_India_ts.values,deaths_India_ts.values, recovered_India_ts.values)
        ] 
    ), 
    index= confirmed_India_ts.index
)

In [ ]:
#Matplotlib
fig=plt.figure(figsize=(10,5),dpi=200)
axes=fig.add_axes([0.1,0.1,0.8,0.8])
axes.bar(dbd_India["Date"],dbd_India["Total Cases"],color='blue')
axes.set_xlabel("Date")
axes.set_ylabel("Total Cases")
axes.set_title("Confirmed cases in India")
plt.show()

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point,Polygon
import descartes


In [ ]:
!pwd
#df=pd.read_csv(https://www.kaggle.com/imdevskp/corona-virus-report#covid19_clean_complete.csv)

In [ ]:
df=pd.read_csv('/content/covid_19_clean_complete.csv')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.groupby('Date')['Confirmed','Deaths','Recovered'].sum()

In [ ]:
df_per_day=df.groupby('Date')['Confirmed','Deaths','Recovered'].max()

In [ ]:
df_per_day['Confirmed'].max()

In [ ]:
df.groupby(['Country/Region'])['Confirmed','Deaths','Recovered']

In [ ]:
df['Country/Region'].value_counts().plot(kind='bar',figsize=(10,10))

In [ ]:
df['Country/Region'].unique()

In [ ]:
gdf01=gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df['Long'],df['Lat']))

In [ ]:
points=[Point(x,y) for x,y in zip(df.Long,df.Lat)]

In [ ]:
points

In [ ]:
gdf03=gpd.GeoDataFrame(df['geometry'])

In [ ]:
gdf01.plot(figsize=(20,10))

In [ ]:
world=gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
ax=world.plot(figsize=(20,10))

In [ ]:
ax.axis('off')

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))

In [ ]:
gdf01.plot(cmap='Purples',ax=ax)

In [ ]:
world.geometry.boundary.plot(color=None,edgecolor='k',linewidth=2,ax=ax)

In [ ]:
fig.show()

In [ ]:
asia=world[world['continent']=='Asia']

In [ ]:
asia.plot()

In [ ]:
gdf03.plot()

In [ ]:
#Removing any region
#world=world[world['name']!='Antarctica']

In [ ]:
world.plot(column='name',cmap='jet')

In [ ]:
world.plot()

In [ ]:
current_crs=world.crs

In [ ]:
world.to_crs(epsg='3857',inplace=True)

In [ ]:
world.plot(figsize=(20,10))

In [ ]:
world['area']=world.area/10000

In [ ]:
world.plot(column='area',cmap='hsv',legend=True,figsize=(20,10))

In [ ]:
gdf03

In [ ]:
gdf03

In [ ]:
df